<h1><u> Imports</h1></u>

In [1]:
# Import necessary libraries
import requests
import json
import numpy as np

<h1><u> CIs' Avergae Length </h1></u>

In [2]:
def read_json_file(file_path):
    """
    This function reads a JSON file from the specified file path and returns its contents.

    :param file_path: str, path to the local JSON file.
    :return: dict, contents of the JSON file.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data


In [3]:
def calculate_confidence_interval_stats_from_file(file_path):
    """
    This function reads a local JSON file, extracts the confidence intervals,
    and calculates the average length and standard deviation of the confidence intervals.

    :param file_path: str, path to the local JSON file.
    :return: tuple, (average length, standard deviation of the confidence intervals).
    """
    # Step 1: Load the JSON data from the local file
    data = read_json_file(file_path)


    # Step 2: Extract confidence intervals
    conf_intervals = []

    # Traverse through the JSON structure to extract intervals
    for key, value in data.items():
        for subkey, interval in value.items():
            if isinstance(interval, list) and len(interval) == 2:
                # Calculate the length of each confidence interval
                conf_intervals.append(interval[1] - interval[0])

    # Step 3: Calculate the average and standard deviation of the confidence intervals' lengths
    if conf_intervals:  # Ensure there are confidence intervals to process
        average_length = np.mean(conf_intervals)
        std_deviation = np.std(conf_intervals)
        return average_length, std_deviation
    else:
        return None, None  # If no valid intervals were found



In [ ]:
# Example usage:
file_path = 'cate_results/cate_conf_intervals_prop.json'  # Replace with the URL of your JSON file
average_length, std_deviation = calculate_confidence_interval_stats_from_file(file_path)

if average_length is not None:
    print(f"Average length of the confidence intervals: {average_length}")
    print(f"Standard deviation of the confidence intervals' lengths: {std_deviation}")
else:
    print("No valid confidence intervals found in the JSON file.")

Average length of the confidence intervals: 0.07702583316533217
Standard deviation of the confidence intervals' lengths: 0.012979706937343786


In [ ]:
# Example usage:
file_path = 'cate_results/cate_conf_intervals_1nn.json'  # Replace with the URL of your JSON file
average_length, std_deviation = calculate_confidence_interval_stats_from_file(file_path)

if average_length is not None:
    print(f"Average length of the confidence intervals: {average_length}")
    print(f"Standard deviation of the confidence intervals' lengths: {std_deviation}")
else:
    print("No valid confidence intervals found in the JSON file.")

Average length of the confidence intervals: 0.07734774272495736
Standard deviation of the confidence intervals' lengths: 0.014298121237266878


In [ ]:
# Example usage:
file_path = 'cate_results/cate_conf_intervals_3nn.json'  # Replace with the URL of your JSON file
average_length, std_deviation = calculate_confidence_interval_stats_from_file(file_path)

if average_length is not None:
    print(f"Average length of the confidence intervals: {average_length}")
    print(f"Standard deviation of the confidence intervals' lengths: {std_deviation}")
else:
    print("No valid confidence intervals found in the JSON file.")

Average length of the confidence intervals: 0.08707736782559115
Standard deviation of the confidence intervals' lengths: 0.01646581619277609


In [ ]:
# Example usage:
file_path = 'cate_results/cate_conf_intervals_5nn.json'  # Replace with the URL of your JSON file
average_length, std_deviation = calculate_confidence_interval_stats_from_file(file_path)

if average_length is not None:
    print(f"Average length of the confidence intervals: {average_length}")
    print(f"Standard deviation of the confidence intervals' lengths: {std_deviation}")
else:
    print("No valid confidence intervals found in the JSON file.")

Average length of the confidence intervals: 0.09307821530958306
Standard deviation of the confidence intervals' lengths: 0.01646603677540706


### Causal Graphs

In [4]:
import pandas as pd
import json

# Load your data
results_df = pd.read_csv('results_final.csv')

# Initialize the dictionary
ci_dict = {}

# Iterate over the rows to populate the dictionary
for _, row in results_df.iterrows():
    treatment1, treatment2 = eval(row['treatment_pair'])  # Convert the pair string to a tuple of integers
    ci_lower, ci_upper = row['CI_lower'], row['CI_upper']
    
    # If treatment1 is not in ci_dict, initialize it
    if treatment1 not in ci_dict:
        ci_dict[treatment1] = {}
        
    # Add the confidence interval as a list under treatment2
    ci_dict[treatment1][treatment2] = [ci_lower, ci_upper]

# Save the dictionary as a JSON file
with open('confidence_intervals.json', 'w') as json_file:
    json.dump(ci_dict, json_file, indent=4)

print("JSON file saved as 'confidence_intervals.json'")


JSON file saved as 'confidence_intervals.json'


In [5]:
# Example usage:
file_path = 'confidence_intervals.json'  # Replace with the URL of your JSON file
average_length, std_deviation = calculate_confidence_interval_stats_from_file(file_path)

if average_length is not None:
    print(f"Average length of the confidence intervals: {average_length}")
    print(f"Standard deviation of the confidence intervals' lengths: {std_deviation}")
else:
    print("No valid confidence intervals found in the JSON file.")

Average length of the confidence intervals: 0.059391261512434165
Standard deviation of the confidence intervals' lengths: 0.028876156349595193


<h1><u> Get Best Month </h1></u>

In [7]:
import json

def compare_months_for_best(data):
    """
    Compare each month's CATE values with every other month's CATE values
    and determine which month beats the most other months.

    :param data: dict, dictionary where keys are months and values are dictionaries of CATE values for other months.
    :return: str, the month that beats the most other months.
    """
    # Initialize a dictionary to track how many months each month beats
    beats_count = {month: 0 for month in data.keys()}
    beats_count['12']=0
    # Get the list of months (keys in the data)
    months = list(data.keys())

    # Compare every pair of months
    for month_i in data.keys():
        for month_j in data[month_i].keys():
            # if i != j:
                # month_i = months[i]
                # month_j = months[j]

                # if month_i in data.keys() and  month_i in data[month_i].keys():
                if data[month_i][month_j] < 0:
                  beats_count[month_j] += 1
                elif data[month_i][month_j] > 0:
                  beats_count[month_i] += 1

    # Find the month that beats the most other months
    best_month = max(beats_count, key=beats_count.get)
    return best_month, beats_count



In [12]:
file_path = "cate_results/cate_results_prop.json"
data = read_json_file(file_path)
# Call the function to find the best month
best_month, beats_count = compare_months_for_best(data)

# Print the result
print(f"The best month is {best_month} with the following number of wins against other months:")
for month, count in beats_count.items():
    print(f"{month}: {count} wins")

The best month is 10 with the following number of wins against other months:
9: 9 wins
11: 10 wins
10: 11 wins
4: 3 wins
6: 6 wins
5: 7 wins
8: 5 wins
3: 2 wins
1: 2 wins
7: 0 wins
2: 5 wins
12: 6 wins


In [ ]:
file_path = "cate_results/cate_results_1nn.json"
data = read_json_file(file_path)
# Call the function to find the best month
best_month, beats_count = compare_months_for_best(data)

# Print the result
print(f"The best month is {best_month} with the following number of wins against other months:")
for month, count in beats_count.items():
    print(f"{month}: {count} wins")

The best month is 10 with the following number of wins against other months:
9: 9 wins
11: 10 wins
10: 11 wins
4: 3 wins
6: 6 wins
5: 7 wins
8: 5 wins
3: 3 wins
1: 2 wins
7: 0 wins
2: 5 wins
12: 5 wins


In [ ]:
file_path = "cate_results/cate_results_3nn.json"
data = read_json_file(file_path)
# Call the function to find the best month
best_month, beats_count = compare_months_for_best(data)

# Print the result
print(f"The best month is {best_month} with the following number of wins against other months:")
for month, count in beats_count.items():
    print(f"{month}: {count} wins")

The best month is 2 with the following number of wins against other months:
9: 3 wins
11: 8 wins
10: 5 wins
4: 2 wins
6: 6 wins
5: 6 wins
8: 6 wins
3: 6 wins
1: 3 wins
7: 4 wins
2: 9 wins
12: 8 wins


In [ ]:
file_path = "cate_results/cate_results_5nn.json"
data = read_json_file(file_path)
# Call the function to find the best month
best_month, beats_count = compare_months_for_best(data)

# Print the result
print(f"The best month is {best_month} with the following number of wins against other months:")
for month, count in beats_count.items():
    print(f"{month}: {count} wins")

The best month is 2 with the following number of wins against other months:
9: 2 wins
11: 5 wins
10: 5 wins
4: 3 wins
6: 5 wins
5: 5 wins
8: 6 wins
3: 7 wins
1: 6 wins
7: 5 wins
2: 9 wins
12: 8 wins


### Causal Graphs

In [9]:
# Initialize the dictionary
ate_dict = {}

# Iterate over the rows to populate the dictionary
for _, row in results_df.iterrows():
    treatment1, treatment2 = eval(row['treatment_pair'])  # Convert the pair string to a tuple of integers
    ate_value = row['ATE']
    
    # If treatment1 is not in ate_dict, initialize it
    if treatment1 not in ate_dict:
        ate_dict[treatment1] = {}
        
    # Add the ATE value under treatment2
    ate_dict[treatment1][treatment2] = ate_value

# Save the dictionary as a JSON file
with open('ate_values.json', 'w') as json_file:
    json.dump(ate_dict, json_file, indent=4)

print("JSON file saved as 'ate_values.json'")


JSON file saved as 'ate_values.json'


In [10]:
file_path = "ate_values.json"
data = read_json_file(file_path)
# Call the function to find the best month
best_month, beats_count = compare_months_for_best(data)

# Print the result
print(f"The best month is {best_month} with the following number of wins against other months:")
for month, count in beats_count.items():
    print(f"{month}: {count} wins")

The best month is 9 with the following number of wins against other months:
9: 10 wins
11: 10 wins
10: 10 wins
4: 4 wins
6: 7 wins
5: 4 wins
8: 8 wins
3: 4 wins
1: 4 wins
7: 1 wins
2: 4 wins
12: 0 wins
